# Calendar Spread Backtest with DoltHub Dataset

This notebook demonstrates backtesting a Calendar Spread strategy using real options data.

## Strategy Overview

**Calendar Spread** (Time Spread) profits from time decay:
- Sell near-term option (high theta decay)
- Buy far-term option (low theta decay)
- Profit from differential time decay
- Typically delta-neutral at initiation

In [ ]:
# Imports
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, str(Path.cwd().parent))

from backtester import (
    DoltHubAdapter,
    MarketDataLoader,
    CalendarSpreadStrategy,
    BacktestEngine,
    BacktestConfig,
    BlackScholesModel,
    PerformanceMetrics,
    VisualizationEngine,
    BreakevenAnalyzer
)

%matplotlib inline

## Step 1: Load Market Data

In [ ]:
# Configuration
DB_PATH = "/Users/janussuk/Desktop/dolt_data/options"
TICKER = "AAPL"
START_DATE = "2024-01-01"
END_DATE = "2024-02-28"

# Load data
adapter = DoltHubAdapter(DB_PATH)
loader = MarketDataLoader(adapter)

market_data = loader.load(
    ticker=TICKER,
    start_date=START_DATE,
    end_date=END_DATE,
    build_vol_surface=True
)

## Step 2: Design Calendar Spread

In [ ]:
# Entry date
entry_date = pd.Timestamp("2024-01-03")
spot = market_data.get_spot(entry_date)

print(f"Entry Date: {entry_date.date()}")
print(f"Spot Price: ${spot:.2f}")

# Get available expirations
expirations = loader.get_available_expirations(TICKER, entry_date.strftime('%Y-%m-%d'))
print(f"\nAvailable Expirations:")
for i, exp in enumerate(expirations[:8]):
    days = (exp - entry_date).days
    print(f"  {i+1}. {exp.date()} ({days} days)")

In [ ]:
# Select expirations (e.g., 30 and 60 days)
near_exp = min(expirations, key=lambda x: abs((x - entry_date).days - 30))
far_exp = min(expirations, key=lambda x: abs((x - entry_date).days - 60))

near_days = (near_exp - entry_date).days
far_days = (far_exp - entry_date).days

print(f"\nNear-term expiration: {near_exp.date()} ({near_days} days)")
print(f"Far-term expiration:  {far_exp.date()} ({far_days} days)")

# ATM strike
strikes = loader.get_strikes_for_expiration(
    TICKER, 
    entry_date.strftime('%Y-%m-%d'),
    near_exp.strftime('%Y-%m-%d')
)
atm_strike = min(strikes, key=lambda x: abs(x - spot))

print(f"ATM Strike: ${atm_strike:.2f}")

In [ ]:
# Create calendar spread
calendar = CalendarSpreadStrategy(
    underlying=TICKER,
    strike=atm_strike,
    near_expiry=near_exp,
    far_expiry=far_exp,
    option_type='call',
    quantity=1.0
)

print(f"\n{calendar}")

## Step 3: Analyze Initial Greeks and Breakevens

In [ ]:
# Calculate initial Greeks
model = BlackScholesModel(use_market_iv=True)
greeks = calendar.greeks(entry_date, market_data, model)

print(f"Initial Greeks:")
print(f"  Delta: {greeks['delta']:8.4f}")
print(f"  Gamma: {greeks['gamma']:8.4f}")
print(f"  Theta: {greeks['theta']:8.4f} (per day)")
print(f"  Vega:  {greeks['vega']:8.4f}")

# Initial value
initial_value = calendar.value(entry_date, market_data, model)
print(f"\nInitial Premium Paid: ${initial_value:,.2f}")

In [ ]:
# Breakeven analysis
analyzer = BreakevenAnalyzer(calendar, market_data, model)
analyzer.print_breakevens(entry_date)

## Step 4: Run Backtest

In [ ]:
# Configure backtest (run until near-term expiration)
config = BacktestConfig(
    start_date=entry_date,
    end_date=near_exp,
    initial_capital=100000.0,
    transaction_cost_per_contract=0.65,
    model=model
)

# Run
engine = BacktestEngine(market_data, config)
engine.add_strategy(calendar, entry_date=entry_date)
results = engine.run()

## Step 5: Performance Analysis

In [ ]:
# Metrics
metrics = PerformanceMetrics(results)
metrics.print_summary()

In [ ]:
# Visualizations
viz = VisualizationEngine(use_plotly=False)

# Equity curve
viz.plot_equity_curve(results, title="Calendar Spread - Portfolio Value")

In [ ]:
# Greeks over time
viz.plot_greeks(results, greeks=['delta', 'theta', 'vega'])

In [ ]:
# Theta decay visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Spot vs strikes
ax1.plot(results.index, results['spot'], 'b-', linewidth=2, label='Spot Price')
ax1.axhline(y=atm_strike, color='r', linestyle='--', label='Strike')
ax1.set_ylabel('Price ($)')
ax1.set_title(f'{TICKER} Price Movement')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Theta over time
ax2.plot(results.index, results['theta'], 'g-', linewidth=2, label='Portfolio Theta')
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Date')
ax2.set_ylabel('Theta ($/day)')
ax2.set_title('Theta Decay Over Time')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

Calendar spreads are excellent strategies for:
- ✓ Profiting from time decay differential
- ✓ Low-risk, defined-risk trades
- ✓ Neutral market outlook
- ✓ Volatility expansion plays

## Experimentation Ideas

- Try different expiration pairs (weekly vs monthly)
- Test on high vs low IV environments
- Compare call vs put calendars
- Implement rolling calendars
- Test OTM vs ATM strikes